In [2]:
import os
import sys
sys.path.append('../../')

from arboretum.core import *
from arboretum.utils import *
from arboretum.algo import *

import matplotlib.pyplot as plt
import distributed

In [3]:
distributed.__version__

'1.21.8'

In [4]:
macosko_40k = '/media/tmo/data/work/datasets/macosko/in/macosko_40k.tsv.gz'
tfs = '/media/tmo/data/work/datasets/macosko/in/mm9_TFs.upper.txt'
out = 'mac_40k.tsv'

In [5]:
from dask.distributed import Client, LocalCluster

In [6]:
client = Client(LocalCluster(memory_limit=8e9))

In [7]:
client

Client Scheduler: tcp://127.0.0.1:43811 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 88 Cores: 88 Memory: 704.00 GB


In [8]:
%%time
macosko_df = pd.read_csv(macosko_40k, sep='\t', dtype='float32')
macosko_df = macosko_df.loc[:, (macosko_df != 0).any(axis=0)]  # remove all-0 columns

CPU times: user 5min 14s, sys: 2min 3s, total: 7min 18s
Wall time: 4min 55s


In [9]:
macosko_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 12934 entries, KITL to 6030443J06RIK
dtypes: float32(12934)
memory usage: 1.9 GB


In [10]:
macosko_df.head()

,KITL,TMTC3,CEP290,4930430F08RIK,1700017N19RIK,RASSF9,LRRIQ1,ADGB,SLC6A15,TMTC2,...,LRRC17,ARMC10,NAPEPLD,PMPCB,DNAJC2,SLC26A5,RELN,GM10475,ORC5,6030443J06RIK
0,0.0,0.000000,4.081807,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,1.976703,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.253301,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,2.658875,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [11]:
mm9_tfs = load_tf_names(tfs)

In [12]:
%%time
network_df = grnboost(expression_data=macosko_df,
                      n_estimators=500,
                      tf_names=mm9_tfs,
                      client_or_address=client,
                      verbose=True)

distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

not shutting down client, client was created externally
finished
CPU times: user 2h 44min 56s, sys: 2h 29min 37s, total: 5h 14min 33s
Wall time: 7h 14min 39s


```
finished
CPU times: user 2h 44min 56s, sys: 2h 29min 37s, total: 5h 14min 33s
Wall time: 7h 14min 39s
```

In [13]:
len(network_df)

4279495

In [32]:
%%time
network_df = grnboost2(expression_data=macosko_df,
                       tf_names=mm9_tfs,
                       client_or_address=client,
                       verbose=True)

distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

not shutting down client, client was created externally
finished
CPU times: user 1h 18min 15s, sys: 58min 48s, total: 2h 17min 4s
Wall time: 2h 37min 55s


```
GRNBoost2 finished (float32 dataframe)
CPU times: user 1h 18min 15s, sys: 58min 48s, total: 2h 17min 4s
Wall time: 2h 37min 55s

GRNBoost2 finished (float64 dataframe)
CPU times: user 1h 17min 14s, sys: 57min 49s, total: 2h 15min 4s
Wall time: 2h 38min 53s
```

In [14]:
len(network_df)

2921508

In [17]:
macosko_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 12953 entries, KITL to 6030443J06RIK
dtypes: float64(12953)
memory usage: 3.9 GB


In [22]:
macosko_df_16 = macosko_df.astype('float16')

In [28]:
macosko_df_16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 12953 entries, KITL to 6030443J06RIK
dtypes: float16(12953)
memory usage: 988.2 MB


In [ ]:
%%time
network_df = grnboost2(expression_data=macosko_df_16,
                       tf_names=mm9_tfs,
                       client_or_address=client,
                       verbose=True)

preparing dask client
parsing input
creating dask graph
computing dask graph
